<a href="https://colab.research.google.com/github/eseylar/PortfolioProjects/blob/main/2022_Music_Wrapped_Metadata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#IMPORTING AND INSTALLING LIBRARIES

#install musicbrainz library (music metadata source)
!pip install musicbrainzngs

#import libraries
import musicbrainzngs as mb
import pandas as pd
import numpy as np
from pandas.core.api import isnull
import json
import time

In [ ]:
#SETTING USER AGENT (app name, version, contact info) -- required to contact api

mb.set_useragent('My 2022 in Music Wrapped', '0.2', contact=None)

In [ ]:
#IMPORTING LISTENING DATA (all 2022 Spotify music streams)
df = pd.read_csv('listening_2022.csv')



---


In [ ]:
#TRUNCATING DATAFRAME

#Note: It takes several hours to execute this code over all 8491 rows in the data frame due to speed limitations set by the MusicBrainz API. 
#      To work around this, I established code to limit the dataframe to 1000 rows at a time to run the code in smaller chunks. At the end, I merge them into one dataframe again.
#      In order to change my set of rows, I just copy/paste the appropriate index into hard brackets on line 20 of this code chunk, 
#      Once I finish exporting a completed datafram, I run this code with the next index of dataframe rows. Rinse and Repeat.

#indexing code chunks
#[0:1000] rows 0-999
#[1000:2000] rows 1000-1999
#[2000:3000] rows 2000-2999
#[3000:4000] rows 3000-3999
#[4000:5000] rows 4000-4999
#[5000:6000] rows 5000-5999
#[6000:7000] rows 6000-6999
#[7000:8000] rows 7000-7999
#[8000:8491] rows 8000-8490

df = pd.read_csv('listening_2022.csv')
df = df.iloc[0:1000] #insert apprporiate chunk inside the hard brackets to the left
df

In [ ]:
#ADDING GENRE AND DURATION DATA TO DATAFRAME


#Counter to run program just once
count = 1 

#Set empty lists for each new piece of info from Music Brainz, to latter be added back to the dataframe
#Original dataframe
genre_metadata = []
dur_metadata = []
artist_id_metadata = []
song_id_metadata = []



#New dataframe for song_id and genre (to make another table)
song_genre_df = pd.DataFrame(columns=['a', 'b']) #new dataframe for song_id and genre (new line for each genre)

#New lists to populate with new information for the genre dataframe
song_id_genre_df = [] #song_id to be added to df
genre_genre_df = [] #genre to be added to df
song_genre_data = [] #empty list to add each loop item to , to later be 


bad_chars = ' ' #space is 'bad character' to be replaced in my genres list; this allows me to later separate a genre string by ' ' delmiiter


while count != 2: #looper running loop just once

    for ind in df.index: #for each row in the original dataframe of my listening history

        artist = df['artist'][ind] #variable artist = the record in the row's artist column
        song = df['song'][ind] #variable song = the record in the row's song column
        album = df['album'][ind] #variable album = the record in the row's album column

        genre_list = [] #set an empty list for genres to go into 
        dur_list = [] # set an empty list for duration
        
        artist_list = mb.search_artists(query=artist)['artist-list'] #query music brain using the artist from the datafram as keyword
        try: #in case of error
            artist_info = artist_list[0] #select the first item from musicbrainz's list, the artist whose name best matches the queried artist name
        except IndexError:
            pass


        #Generate artist_id
        try: #incase of artist_info error, this try block will add a null to the relevant metadata rather than stop the code with an error
            artist_id = artist_info['id']
        except:
            artist_id = None

        artist_id_metadata.append(artist_id)


        #Generate duration
        recording_list = mb.search_recordings(query=song, limit=1, offset=None, strict=False, artist=artist, release=album) #search for the track in musicbrainz library
        try: #if duration is available, pull duration in milliseconds; if not, set dur to None-type
            dur = recording_list['recording-list'][0]['length'] #select the length in milliseconds of the track
            dur = int(dur) / 1000 #duration in seconds, rather than milliseconds
        except:
            dur = None

        dur_metadata.append(dur)


        #Generate song_id
        song_id = recording_list['recording-list'][0]['id']

        song_id_metadata.append(song_id)


        #Generate genre list
        genre_counter = 0 #counter so script can query a variable amount of genres
        try:
            for item in artist_info['tag-list']: #for each genre tagged in the artist profile...
                
                if int(artist_info['tag-list'][genre_counter]['count']) > 0: #if the 'count' element is greater than 0 (basically, if people haven't downvoted the tag)...
                    new_genre = artist_info['tag-list'][genre_counter]['name'] #set that genre's name to the variable new_genre...
                                  
                    #replace ' ' with '-'
                    for i in bad_chars:
                        new_genre = new_genre.replace(i, '-')


                    #Generating song_id and genre for the genre table
                    song_id_genre_df.append(song_id) #add song_id to the list 
                    genre_genre_df.append(new_genre) #add the new genre to the list

                    song_genre_data.append([song_id, new_genre]) #add new row of data with song_id and genre
                    
                    
                    #Adding genre to a list for the original dataframe
                    genre_list.append(new_genre) #add the new genre to the list
                    genre_counter += 1 #add one to the counter to move to next item in 'tag-list'
                else: #if the 'count' is 0 or negative....
                    genre_counter += 1 #add one to the counter to move to next item in 'tag-list', don't save that genre to list
        except:
            new_genre = None #if there are no genre tags, set the genre to None-type
            genre_list.append(new_genre)

        genre_metadata.append(genre_list)


        time.sleep(0.02) #added delay; Musicbrainz throttles queries past 50 per second

    count += 1 #add one to overall counter/looper

#Adding blank columns to the scrobble dataframe (original dataset)
df["genre"] = np.nan #create a new data frame column called genre, fill it with NaN-type
df["duration"] = np.nan #create a new data frame column called duration, fill it with NaN-type
df["artist_id"] = np.nan #create a new data frame column called artist_id, fill it with NaN-type
df["song_id"] = np.nan #create a new data frame column called artist_id, fill it with NaN-type

#Add series data to the scrobble dataframe, overwriting blank columns just added
df = df.assign(genre = genre_metadata) # fill the genre column with the genre data previously generated
df = df.assign(duration = dur_metadata) # fill the duration column with the duration data previously generated
df = df.assign(artist_id = artist_id_metadata) # fill the aritst_id column with the artist_id data previously generated
df = df.assign(song_id = song_id_metadata) # fill the song_id column with the song_id data previously generated

#Add song_id and genre to the genre dataframe (then do some work to rename and reorganinze dataframe, which wouldn't let me append properly)
song_genre_df = song_genre_df.append(song_genre_data, )
song_genre_df.rename( columns={0 :'song_id'}, inplace=True )
song_genre_df.rename( columns={1 :'genre'}, inplace=True )
song_genre_df.drop(['a', 'b'], axis=1, inplace=True)

display(df) #Show original scrobble dataframe
display(song_genre_df) #Show song_id and genre dataframe

In [ ]:
#EXPORTING DATAFRAMES AS .CSV FILES
#Note: Since I ran this code in small chunks of 1000 rows, I needed code to export each dataframe with a different naming schemea.
#      To execute, comment all but the appropriate code chunk and run the cell. 

#Code Chunk no. 1 -- rows 0-999 (index 0:1000)
df.to_csv('scrobbles_0_999.csv', index=False) #scrobble dataframe - rows 0-999
song_genre_df.to_csv('genres_0_999.csv', index=False) #genre table dataframe rows 0-999

#Code Chunk no. 2 -- rows 1000-1999 (index 1000:2000)
df.to_csv('scrobbles_1000_1999.csv', index=False) #scrobble dataframe - rows 1000-1999
song_genre_df.to_csv('genres_1000_1999.csv', index=False) #genre table dataframe rows 1000-1999

#Code Chunk no. 3 -- rows 2000-2999 (index 2000:3000)
df.to_csv('scrobbles_2000_2999.csv', index=False) #scrobble dataframe - rows 2000-2999
song_genre_df.to_csv('genres_2000_2999.csv', index=False) #genre table dataframe rows 2000-2999

#Code Chunk no. 4 -- rows 3000-3999 (index 3000:4000)
df.to_csv('scrobbles_3000_3999.csv', index=False) #scrobble dataframe - rows 3000-3999
song_genre_df.to_csv('genres_3000_3999.csv', index=False) #genre table dataframe rows 3000-3999

#Code Chunk no. 5 -- rows 4000-4999 (index 4000:5000)
df.to_csv('scrobbles_4000_4999.csv', index=False) #scrobble dataframe - rows 4000-4999
song_genre_df.to_csv('genres_4000_4999.csv', index=False) #genre table dataframe rows 4000-4999

#Code Chunk no. 6 -- rows 5000-5999 (index 5000:6000)
df.to_csv('scrobbles_5000_5999.csv', index=False) #scrobble dataframe - rows 5000-5999
song_genre_df.to_csv('genres_5000_5999.csv', index=False) #genre table dataframe rows 5000-5999

#Code Chunk no. 7 -- rows 6000-6999 (index 6000:7000)
df.to_csv('scrobbles_6000_6999.csv', index=False) #scrobble dataframe - rows 6000-6999
song_genre_df.to_csv('genres_6000_6999.csv', index=False) #genre table dataframe rows 6000-6999

#Code Chunk no. 8 -- rows 7000-7999 (index 7000:8000)
df.to_csv('scrobbles_7000_7999.csv', index=False) #scrobble dataframe - rows 7000-7999
song_genre_df.to_csv('genres_7000_7999.csv', index=False) #genre table dataframe rows 7000-7999

#Code Chunk no. 9 -- rows 8000-8490 (index 8000:8490)
df.to_csv('scrobbles_8000_8490.csv', index=False) #scrobble dataframe - rows 8000-8490
song_genre_df.to_csv('genres_8000_8490.csv', index=False) #genre table dataframe rows 8000-8490

## Concatenating Smaller Dataframes

In [ ]:
#CONCAT FULL SCROBBLE DATA

df0 = pd.read_csv('scrobbles_0_999.csv')
df1000 = pd.read_csv('scrobbles_1000_1999.csv')
df2000 = pd.read_csv('scrobbles_2000_2999.csv')
df3000 = pd.read_csv('scrobbles_3000_3999.csv')
df4000 = pd.read_csv('scrobbles_4000_4999.csv')
df5000 = pd.read_csv('scrobbles_5000_5999.csv')
df6000 = pd.read_csv('scrobbles_6000_6999.csv')
df7000 = pd.read_csv('scrobbles_7000_7999.csv')
df8000 = pd.read_csv('scrobbles_8000_8490.csv')

full_scrobble_df = pd.concat([df0, df1000, df2000, df3000, df4000, df5000, df6000, df7000, df8000], axis=0, ignore_index=True)

full_scrobble_df.to_csv('full_scrobble_data_2022.csv', index=False)

In [ ]:
#CONCAT GENRE TABLE

g0 = pd.read_csv('genres_0_999.csv')
g1000 = pd.read_csv('genres_1000_1999.csv')
g2000 = pd.read_csv('genres_2000_2999.csv')
g3000 = pd.read_csv('genres_3000_3999.csv')
g4000 = pd.read_csv('genres_4000_4999.csv')
g5000 = pd.read_csv('genres_5000_5999.csv')
g6000 = pd.read_csv('genres_6000_6999.csv')
g7000 = pd.read_csv('genres_7000_7999.csv')
g8000 = pd.read_csv('genres_8000_8490.csv')

full_genre_df = pd.concat([g0, g1000, g2000, g3000, g4000, g5000, g6000, g7000, g8000], axis=0, ignore_index=True)

full_genre_df.to_csv('full_genre_data_2022.csv', index=False)